In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI TabNet

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/tabnet_vertex_tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/tabnet_vertex_tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabnet/tabnet_vertex_tutorial.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial demonstrates how to use the TabNet algorithm on the Vertex AI platform.

TabNet combines the best of two worlds: it is explainable (similar to simpler tree-based models) while benefiting from high performance (similar to deep neural networks). This makes it great for retailers, finance and insurance industry applications such as predicting credit scores, fraud detection and forecasting. 

TabNet uses a machine learning technique called sequential attention to select which model features to reason from at each step in the model. This mechanism makes it possible to explain how the model arrives at its predictions and helps it learn more accurate models. Thanks to this design, TabNet not only outperforms other neural networks and decision trees but also provides interpretable feature attributions. Releasing TabNet as a First Party Trainer in Vertex AI means you'll be able to easily take advantage of TabNet's architecture and explainability and use it to train models on your own data. 

Learn more about [Tabular Workflow for TabNet](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/tabnet).

### Objective

In this notebook, you learn how to run TabNet model on Vertex AI.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Training
- Vertex AI Hyperparameter Tuning
- TabNet builtin algorithm
- BigQuery

The steps performed are:
1. **Setup**: Importing the required libraries and setting your global variables.
2. **Configure parameters**: Setting the appropriate parameter values for the training job.
3. **Train on Vertex AI Training**: Submitting a training job using csv input.
4. **Hyperparameter tuning**: Running a hyperparameter tuning job.
5. **Hyperparameter on Vertex AI Training with BigQuery input**: Submitting a training job using BigQuery input.
6. **Cleaning up**: Deleting resources created by this tutorial.

### Dataset

This tutorial uses the `petfinder` in the public bucket `gs://cloud-samples-data/ai-platform-unified/datasets/tabular/`, which was generated from the [PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction). 

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

In [ ]:
! pip3 install --quiet --upgrade tensorflow
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 tensorboard-plugin-profile \
                                 google-cloud-pipeline-components
! gcloud components update --quiet

### Colab only: Uncomment the following cell to restart the kernel.


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the following APIs: Vertex AI API, Cloud Resource Manager API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI Model resource and use for prediction.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries and define constants

In [ ]:
import os
import re
import time
from datetime import datetime

from google.cloud import aiplatform

%load_ext tensorboard

In [ ]:
# Please note that if you use csv input, the first column is the label column.

IMPORT_FILE = "petfinder-tabular-classification-tabnet-with-header.csv"
! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {BUCKET_URI}/data/petfinder/train.csv

gcs_source = f"{BUCKET_URI[5:]}/data/petfinder/train.csv"
print("gcs_source: ", gcs_source)

## Configure parameters

The following table shows parameters that are common to all Vertex Training jobs created using the `gcloud ai custom-jobs create` command. See the [official documentation](https://cloud.google.com/sdk/gcloud/reference/ai/custom-jobs/create) for all the possible arguments.

| Parameter | Data type | Description | Required |
|--|--|--|--|
| `display-name` | string | Name of the job. | Yes |
| `worker-pool-spec` | string | Comma-separated list of arguments specifying a worker pool configuration (see below). | Yes |
| `region` | string | Region to submit the job to. | No |

The `worker-pool-spec` flag can be specified multiple times, one for each worker pool. The following table shows the arguments used to specify a worker pool.

| Parameter | Data type | Description | Required |
|--|--|--|--|
| `machine-type` | string | Machine type for the pool. See the [official documentation](https://cloud.google.com/vertex-ai/docs/training/configure-compute) for supported machines. | Yes |
| `replica-count` | int | The number of replicas of the machine in the pool. | No |
| `container-image-uri` | string | Docker image to run on each worker. | No |

The following table shows the parameters for the TabNet training job:

| Parameter | Data type | Description | Required |
|--|--|--|--|
| `preprocess` | boolean argument | Specify this to enable automatic preprocessing. | No|
| `job_dir` | string | Cloud Storage directory where the model output files will be stored. | Yes |
| `input_metadata_path` | string | The GCS path to the TabNet-specific metadata for the training dataset. Please see above on how to create the metadata. | No . |
| `training_data_path` | string | Cloud Storage pattern where training data is stored. | Yes |
| `validation_data_path` | string | Cloud Storage pattern where eval data is stored. | No |
| `test_data_path` | string | Cloud Storage pattern where test data is stored. | Yes |
| `input_type` | string | “bigquery“ or “csv“ - type of the input tabular data. If csv is mentioned then the first column is treated as target. If CSV files have a header, also pass the flag “data_has_header”. If “bigquery” is used, one can either supply training/validation data paths, or supply BigQuery project, dataset, and table names for preprocessing to produce the training and validation datasets.. | No -Deafult is "csv" |
| `model_type` | string | The learning task such as classification or regression. | Yes |
| `split_column` | string | The column name used to create the training, validation, and test splits. Values of the columns (a.k.a table['split_column']) should be either “TRAIN”, “VALIDATE”, or “TEST”. “TEST” is optional. Applicable to bigquery input only. | No. |
| `train_batch_size` | int | Batch size for training. | No - Default is 1024. |
| `eval_split` | float | Split fraction to use for the evaluation dataset, if `validation_data_path` is not provided. | No - Default is 0.2 |
| `learning_rate` | float | Learning rate for training. | No - Default is the default learning rate of the specified optimizer. |
| `eval_frequency_secs` | int | Frequency at which evaluation and checkpointing will take place.The default is 600. | No . |
| `num_parallel_reads` | int | Number of threads used to read input files. We suggest setting it equal or slightly less than the number of CPUs of the machine for maximal performance in most cases. For example, 6 per GPU is a good default choice. | Yes . |
| `data_cache` | string | Choose to cache data to “memory”, “disk” or “no_cache”. For large datasets, caching the data into memory would throw out-of-memory errors, therefore, we suggest choosing “disk”. You can specify the disk size in the config file (as exemplified below). Make sure to request a sufficiently large (e.g. TB size) disk to write the data, for large (B-scale) datasets. | No, The default one is “memory” . |
| `bq_project` | string | The name of the BigQuery project. If input_type=bigquery and using the flag –preprocessing, this is required. This is an alternative to specifying train, validation, and test data paths. | No . |
| `dataset_name` | string | The name of the BigQuery dataset. If input_type=bigquery and using the flag –preprocessing, this is required. This is an alternative to specifying train, validation, and test data paths. | No . |
| `table_name` | string | The name of the BigQuery table. If input_type=bigquery and using the flag –preprocessing, this is required. This is an alternative to specifying train, validation, and test data paths. | No . |
| `loss_function_type` | string | There are several loss function types in TabNet. For regression: mse/mae are included. For classification: cross_entropy/weighted_cross_entropy/focal_loss are included | No . If the value is "default", we use mse for regression and cross_entropy for classification.|
| `deterministic_data` | boolean argument | Determinism of data reading from tabular data. The default is set to False. When setting to True, the experiment is deterministic. For fast training on large datasets, we suggest deterministic_data=False setting, albeit having randomness in the results (which becomes negligible at large datasets). Note that determinism is still not guaranteed with distributed training, as map-reduce causes randomness due to the ordering of algebraic operations with finite precision. However, this is negligible in practice, especially on large datasets. For the cases where 100% determinism is desired, besides deterministic_data=True setting, we suggest training with a single GPU (e.g. with MACHINE_TYPE="n1-highmem-8").| No, The default is False . |
| `stream_inputs` | boolean argument | Stream input data from GCS instead of downloading it locally - this option is suggested for fast runtime. | No . |
| `large_category_dim` | int | Dimensionality of the embedding - if the number of distinct categories for a categorical column is greater than large_category_thresh we use a large_category_dim dimensional embedding, instead of 1-D embedding. The default is 1. We suggest increasing it (e.g. to ~5 in most cases and even ~10 if the number of categories is typically very large in the dataset), if pushing the accuracy is the main goal, rather than computational efficiency and explainability.  | No . |
| `large_category_thresh` | int | Threshold for categorical column cardinality - if the number of distinct categories for a categorical column is greater than large_category_thresh we use a large_category_dim dimensional embedding, instead of 1-D embedding. The default is 300. We suggest decreasing it (e.g. to ~10), if pushing the accuracy is the main goal, rather than computational efficiency and explainability. | No . |
| `yeo_johnson_transform` | boolean argument | Enables trainable Yeo-Johnson Power Transform (the default is disabled). Please see this link: https://www.stat.umn.edu/arc/yjpower.pdf for more information on Yeo-Johnson Power Transform. With our implementation, the transform parameters are learnable along with TabNet, trained in an end-to-end way. | No . |
| `apply_log_transform` | boolean argument | If log transform statistics are contained in the metadata and this flag is true then the input features will be log transformed. Use false to not use the transform and true (the default) to use it. Especially for datasets with skewed numerical distributions, log transformations could be very helpful.  | No . |
| `apply_quantile_transform` | boolean argument | If quantile statistics are contained in the metadata and this flag is true then the input features will be quantile transformed. Use false to not use the transform and true (the default) to use it. Especially for datasets with skewed numerical distributions, quantile transformations could be very helpful. Currently supported for BigQuery input_type. | No . |
| `replace_transformed_features` | boolean argument | If true then if a transformation is applied to a feature that feature will be replaced. If false (the default option) then the transformed feature 'will be added as a new feature to the list of feature columns. Use true to replace the feature with the transform and false to append the transformed feature as a new column. | No . |
| `target_column` | string | name of the label column. Note that for classification, the label needs to be of String or integer type. | No . |
| `prediction_raw_inputs` | boolean argument | If we set this argument, the model serving allows us to pass the features as a dictionary of tensors instead of CSV row. | No . |
| `exclude_key` | boolean argument | If we set this argument, we exclude a key in the input/output. The key is helpful in batch prediction processes input and saves output in an unpredictable order. The key helps match the output with input. | No . |

In [ ]:
# URI of the TabNet training Docker image.
LEARNER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/tab_net_v2"
)

DATASET_NAME = "petfinder"  # Change to your dataset name.

ALGORITHM = "tabnet"
MODEL_TYPE = "classification"
MODEL_NAME = f"{DATASET_NAME}_{ALGORITHM}_{MODEL_TYPE}"

TRAINING_DATA_PATH = f"{BUCKET_URI}/data/{DATASET_NAME}/train.csv"

print("training path: ", TRAINING_DATA_PATH)
print("model name: ", MODEL_NAME)

# Run TabNet training

In [ ]:
current_time = datetime.utcnow().strftime('%y%m%d%H%M%S')
OUTPUT_DIR = f"{BUCKET_URI}/{MODEL_NAME}_{current_time}"
print('Output dir: ', OUTPUT_DIR)
learning_job_name = f'tab_net_{DATASET_NAME}_{current_time}'
print('Job name: ', learning_job_name)

CREATION_LOG = !gcloud ai custom-jobs create \
  --region={REGION} \
  --display-name={learning_job_name} \
  --worker-pool-spec=machine-type=n1-standard-8,replica-count=1,container-image-uri={LEARNER_IMAGE_URI} \
  --args=--preprocess \
  --args=--prediction_raw_inputs \
  --args=--exclude_key \
  --args=--data_has_header \
  --args=--training_data_path={TRAINING_DATA_PATH} \
  --args=--job-dir={OUTPUT_DIR} \
  --args=--model_type={MODEL_TYPE} \
  --args=--max_steps=2000 \
  --args=--batch_size=4096 \
  --args=--learning_rate=0.01

print(CREATION_LOG)

After the job is submitted successfully, you can view its details and logs:

In [ ]:
JOB_ID = re.search(r"(?<=/customJobs/)\d+", CREATION_LOG[1]).group(0)
print(JOB_ID)

In [ ]:
# View the job's configuration and state.
STATE = "state: JOB_STATE_PENDING"

while STATE not in ["state: JOB_STATE_SUCCEEDED", "state: JOB_STATE_FAILED"]:
    DESCRIPTION = ! gcloud ai custom-jobs describe {JOB_ID} --region={REGION}
    STATE = DESCRIPTION[-2]
    print(STATE)
    time.sleep(60)

# Deploy on Vertex AI Prediction

## Import the model
Your training job exports two TF SavedModel under `gs://<job_dir>/model`. The exported model can be used for online or batch prediction in Vertex AI Prediction. First, import the model:

In [ ]:
# Imports the model.
current_time = datetime.utcnow().strftime("%y%m%d%H%M%S")
MODEL_TYPE = "classification"

DISPLAY_NAME = f"tabnet_{DATASET_NAME}_{MODEL_TYPE}_{current_time}"  # The display name of the model.
MODEL_NAME = f"tabnet_{MODEL_TYPE}_model"  # Used by the deployment container.
ARTIFACT_URI = f"{OUTPUT_DIR}/model/"

print("Display_name:", DISPLAY_NAME)
print("Model name: ", MODEL_NAME)

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
)

model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest",
)
print(model)

## Deploy the model
After importing the model, you must deploy it to an endpoint so that you can get online predictions. More information about this process can be found in the [official documentation](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api).

## Create a model endpoint

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=DATASET_NAME)

# If you want to use existing endpoint
# endpoint = aiplatform.Endpoint('[location of existing endpoint]')
print("Endpoint: ", endpoint)

## Deploy model to the endpoint

In [ ]:
model.deploy(
    endpoint=endpoint,
    machine_type="n1-standard-4",
    traffic_split={"0": 100},
    deployed_model_display_name=DISPLAY_NAME,
)

## Online prediction

In [ ]:
prediction = endpoint.predict(
    [
        {
            "Type": "Cat",
            "Age": 3,
            "Breed1": "Tabby",
            "Gender": "Male",
            "Color1": "Black",
            "Color2": "White",
            "MaturitySize": "Small",
            "FurLength": "Short",
            "Vaccinated": "No",
            "Sterilized": "No",
            "Health": "Healthy",
            "Fee": 100,
            "PhotoAmt": 2,
        }
    ]
)

print(prediction)

# Hyperparameter Tuning
After successfully training your model, deploying it, and calling it to make predictions, you may want to optimize the hyperparameters used during training to improve your model's accuracy and performance. See the Vertex AI documentation for an overview of hyperparameter tuning and how to use it in your Vertex Training jobs.

For this example, the following command runs a Vertex AI hyperparameter tuning job with 4 trials that attempts to maximize the validation AUC metric. The hyperparameters it optimizes are the number of max_steps and the learning rate.

In [ ]:
DATASET_NAME = "petfinder"  # Change to your dataset name.
current_time = datetime.utcnow().strftime('%y%m%d%H%M%S')
OUTPUT_DIR = f"{BUCKET_URI[5:]}/hptuning/{MODEL_NAME}_{current_time}/"
print('Output dir: ', OUTPUT_DIR)
learning_job_name = f'tab_net_{DATASET_NAME}_{current_time}'
print('Job name: ', learning_job_name)

config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-highmem-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 100
    containerSpec:
      imageUri: {LEARNER_IMAGE_URI}
      args:
      - --preprocess 
      - --prediction_raw_inputs
      - --exclude_key
      - --data_has_header
      - --training_data_path={TRAINING_DATA_PATH}
      - --job-dir={OUTPUT_DIR}
      - --batch_size=1028
      - --model_type=classification
"""

!echo $'{config}' > ./config.yaml

MAX_TRIAL_COUNT=4
PARALLEL_TRIAL_COUNT=2

!gcloud ai hp-tuning-jobs create \
  --config=config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name=$learning_job_name

# Hyperparameter Tuning with Big Query (BQ) input
You need to create the BQ dataset using the above csv file.

Create BQ dataset to your project


In [ ]:
! bq --location={REGION} mk --dataset {PROJECT_ID}:{DATASET_NAME}

Create the BQ table using the csv file.

In [ ]:
!bq --location={REGION} load --source_format=CSV --autodetect {PROJECT_ID}:{DATASET_NAME}.train {BUCKET_URI}/data/petfinder/train.csv

Run hp-tuning using BQ input.

In [ ]:
current_time = datetime.utcnow().strftime('%y%m%d%H%M%S')

BQ_PROJECT=f"{PROJECT_ID}"
DATASET_NAME = "petfinder"  # Change to your dataset name.
TABLE_NAME="train"
OUTPUT_DIR = f"{BUCKET_URI}/hptuning/{MODEL_NAME}_{current_time}/"
print('Output dir: ', OUTPUT_DIR)
learning_job_name = f'tab_net_{DATASET_NAME}_{current_time}'
print('Job name: ', learning_job_name)

config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-highmem-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 200
    containerSpec:
      imageUri: {LEARNER_IMAGE_URI}
      args:
      - --eval_frequency_secs=10800
      - --input_type=bigquery
      - --preprocess
      - --prediction_raw_inputs
      - --exclude_key
      - --model_type=classification
      - --stream_inputs
      - --bq_project={BQ_PROJECT}
      - --dataset_name={DATASET_NAME}
      - --table_name={TABLE_NAME}
      - --target_column=Adopted
      - --num_parallel_reads=2
      - --optimizer_type=adam
      - --data_cache=disk
      - --deterministic_data=False
      - --loss_function_type=weighted_cross_entropy
      - --replace_transformed_features=True
      - --apply_quantile_transform=True
      - --apply_log_transform=True
      - --batch_size=32768
      - --job-dir={OUTPUT_DIR}
"""

!echo $'{config}' > ./hptuning-config.yaml

MAX_TRIAL_COUNT=2
PARALLEL_TRIAL_COUNT=2

!gcloud ai hp-tuning-jobs create \
  --config=hptuning-config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name=$learning_job_name

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

# Delete BQ table
! bq rm -f {PROJECT_ID}:{DATASET_NAME}.train

# Delete endpoint resource
endpoint.delete(force=True)

# Delete model resource
model.delete()

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI